In [19]:
import os

downloads_path = os.path.expanduser("~/Downloads")
os.chdir(downloads_path)

In [20]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

hvacdata = pd.read_csv('hvac_merged.csv')


/var/folders/9f/3yz89nb525nczbywh5dl11kh0000gn/T/ipykernel_32452/2688843383.py:6: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  hvacdata = pd.read_csv('hvac_merged.csv')


In [36]:
print(hvacdata.columns.tolist())

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)


print(hvacdata.head(5))



['uic', 'job_seq', 'apl', 'cause_code', 'cdm_rin', 'csmp_narrative_summary', 'date_closing', 'date_maintenance_action', 'deferral_reason_code', 'due_date', 'eic', 'equipment_nomenclature', 'eswbs_opening', 'iuc_screening_code', 'location', 'mhc', 'mho', 'mhr', 'priority_code', 'safety_code', 'status_code', 'tycom_screening_code', 'type_availability_code', 'type_of_maintenance_action', 'when_discovered_code', 'work_center', 'total_ima_man_hours', 'total_ship_force_man_hours', 'total_replacement_cost', 'total_repair_replacement_cost', 'def_narrative', 'nondef_narrative', 'closing_narrative', 'ima_narrative', 'total_material_cost', 'opening_ship_system', 'job_status', 'days_open', 'action_taken', 'intermediate_unit_commander_screening', 'priority', 'type_of_availability_needed', 'tycom_screening', 'when_discovered', 'cause', 'deferral_reason', 'feasible_with_ship_inventory', 'feasible_with_additional_dla_inventory', 'ship', 'uic_s', 'jsn', 'date_closing_s', 'demand_date', 'issue_date', 'c

In [22]:


hvacdata['date_maintenance_action'] = pd.to_datetime(hvacdata['date_maintenance_action'], errors='coerce')

# group by `ship` to find the earliest date
rcoh_start_dates = (
    hvacdata.groupby('ship')['date_maintenance_action']
    .min()
    .reset_index()
    .rename(columns={'date_maintenance_action': 'rcoh_start_date'})
)

print(rcoh_start_dates)



# !!!!! RCOH for CVN 68 took place between 1998 and 2001, but the first maintenace job recorded in the dataset for CVN 68 is in 2017?
# Just use start dates within the data? This may not accurately represent the true project phases

     ship rcoh_start_date
0   CVN68      2017-11-02
1   CVN69      2017-11-01
2   CVN70      2017-11-01
3   CVN71      2017-11-04
4   CVN72      2017-11-01
5   CVN73      2017-11-02
6   CVN74      2017-11-01
7   CVN75      2017-11-02
8   CVN76      2017-11-01
9   CVN77      2017-11-01
10  CVN78      2017-11-01


In [23]:
hvacdata = hvacdata.merge(rcoh_start_dates, on='ship', how='left')


In [24]:
# calculate time since RCOH start

hvacdata['months_since_rcoh_start'] = (
    (hvacdata['date_maintenance_action'] - hvacdata['rcoh_start_date']) / np.timedelta64(1, 'M')
).astype(int)


In [25]:
print(hvacdata[['ship', 'date_maintenance_action', 'rcoh_start_date', 'months_since_rcoh_start']].head())

    ship date_maintenance_action rcoh_start_date  months_since_rcoh_start
0  CVN69              2018-07-16      2017-11-01                        8
1  CVN69              2018-07-16      2017-11-01                        8
2  CVN77              2018-07-16      2017-11-01                        8
3  CVN75              2018-05-11      2017-11-02                        6
4  CVN75              2018-05-11      2017-11-02                        6


In [26]:
# group by ship and project phase (months since start)
job_summary = (
    hvacdata.groupby(['ship', 'months_since_rcoh_start'])
    .agg(
        num_jobs=('ship', 'count'),
        total_material_cost=('total_material_cost', 'sum')
    )
    .reset_index()
)

print(job_summary.head(10))


    ship  months_since_rcoh_start  num_jobs  total_material_cost
0  CVN68                        0       109           -330796.57
1  CVN68                        1        23              9950.10
2  CVN68                        2       268           -196928.14
3  CVN68                        3       119            -47056.37
4  CVN68                        4        68           -287370.61
5  CVN68                        5       165           -186002.60
6  CVN68                        6        64             -1804.15
7  CVN68                        7       107            400465.12
8  CVN68                        8        27           -104921.24
9  CVN68                        9        61           -160909.77


In [28]:
wrong_costs = hvacdata[hvacdata['total_material_cost'] <= -10000]
print(bad_costs[['ship', 'total_material_cost', 'opening_ship_system', 'job_status']])
print(f"Number of rows with wrong costs: {wrong_costs.shape[0]}")


        ship  total_material_cost opening_ship_system job_status
15     CVN72             -10000.0         R2129751211     Closed
39     CVN75             -10000.0         R2185351211     Closed
40     CVN70             -10000.0         R2099351211     Closed
41     CVN70             -10000.0         R2099351211     Closed
42     CVN75             -10000.0         R2185343311     Closed
...      ...                  ...                 ...        ...
68209  CVN72             -10000.0         R2129751211       Open
68210  CVN71             -10000.0         R2124751211       Open
68211  CVN72             -10000.0         R2129743311     Closed
68212  CVN78             -10000.0         R2317351211       Open
68213  CVN75             -10000.0         R2185351312       Open

[33383 rows x 4 columns]
Number of rows with wrong costs: 33383


In [29]:
print(hvacdata.columns.tolist())


['uic', 'job_seq', 'apl', 'cause_code', 'cdm_rin', 'csmp_narrative_summary', 'date_closing', 'date_maintenance_action', 'deferral_reason_code', 'due_date', 'eic', 'equipment_nomenclature', 'eswbs_opening', 'iuc_screening_code', 'location', 'mhc', 'mho', 'mhr', 'priority_code', 'safety_code', 'status_code', 'tycom_screening_code', 'type_availability_code', 'type_of_maintenance_action', 'when_discovered_code', 'work_center', 'total_ima_man_hours', 'total_ship_force_man_hours', 'total_replacement_cost', 'total_repair_replacement_cost', 'def_narrative', 'nondef_narrative', 'closing_narrative', 'ima_narrative', 'total_material_cost', 'opening_ship_system', 'job_status', 'days_open', 'action_taken', 'intermediate_unit_commander_screening', 'priority', 'type_of_availability_needed', 'tycom_screening', 'when_discovered', 'cause', 'deferral_reason', 'feasible_with_ship_inventory', 'feasible_with_additional_dla_inventory', 'ship', 'uic_s', 'jsn', 'date_closing_s', 'demand_date', 'issue_date', 'c

In [31]:
job_summary = (
    hvacdata.groupby(['ship', 'months_since_rcoh_start'])
    .agg(
        num_jobs=('ship', 'count'),
        totMaterialCost=('totMaterialCost', 'sum')
    )
    .reset_index()
)

print(job_summary.head(10))


    ship  months_since_rcoh_start  num_jobs  totMaterialCost
0  CVN68                        0       109          3510.49
1  CVN68                        1        23          4342.57
2  CVN68                        2       268         60903.65
3  CVN68                        3       119          8226.57
4  CVN68                        4        68          1608.14
5  CVN68                        5       165          9679.50
6  CVN68                        6        64         26928.51
7  CVN68                        7       107         32103.26
8  CVN68                        8        27           597.35
9  CVN68                        9        61          8536.91


In [35]:
hvacdata['computed_totMaterialCost'] = hvacdata['unit_price'] * hvacdata['quantity']

# compare with the original
print(hvacdata[['totMaterialCost', 'computed_totMaterialCost']].head(10))


   totMaterialCost  computed_totMaterialCost
0            50.92                     50.92
1            10.35                     10.35
2            10.35                     10.35
3           249.98                     20.00
4           249.98                     16.00
5           249.98                     20.00
6           249.98                    160.98
7           249.98                     17.00
8           249.98                     16.00
9           363.59                    363.59
